In [ ]:
from bins import Bins
from utils import calcAllChi2, calcOneChi2, makeHists
from ROOT import TFile, TH1

bins = Bins.readFrom("ranges.yml")
histsMC = makeHists("medium_isotropic_eff_ag1230ag_nn_9deg.dat", "_MC", bins)
histsData = makeHists("apr12_diele_088_090_ag123ag_2500A_accepted_nn_2.dat", "_data", bins)

#outfile = TFIle("out.root","RECREATE")
allHistsMC = []

calcAllChi2(histsMC, histsData)

In [ ]:
len(histsData[0])

In [ ]:
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

In [ ]:
TH1.SetDefaultSumw2

N_PARAMS = 3

grid_x = torch.linspace(-1,1,101)

grid_x1, grid_x2, grid_x3 = torch.meshgrid(grid_x, grid_x, grid_x, indexing="ij")

xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten(), grid_x3.flatten()]).transpose(-2,-1)

print(xs)
print(xs.size())

In [ ]:
lb = -1
ub = 1

bounds = torch.tensor([[lb]*N_PARAMS, [ub]*N_PARAMS], dtype=torch.float)
bounds

In [ ]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [ ]:
num_queries = 20
num_repeats = 1

In [ ]:
def lambdas(x):
    return x[0][0], x[0][1], x[0][2]

def cost(x):
    lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

    result = 0

    if pow(1-lambda_phi, 2) - pow(lambda_theta-lambda_phi, 2) < 4*pow(lambda_theta_phi, 2):
        result = 1
    if 1 + lambda_theta + 2*lambda_phi < 0:
        result = 1

    return torch.tensor([result])

def objective(x):
    lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

    histsMC = makeHists("medium_isotropic_eff_ag1230ag_nn_9deg.dat", "_MC", bins, lambda_theta, lambda_phi, lambda_theta_phi)
    chi2, ndf = calcOneChi2(histsMC[0][0], histsData[0][0])
    allHistsMC.append(histsMC[0][0])
    if not chi2 or not ndf:
        return torch.tensor([0])
    return torch.tensor([1./(chi2 / ndf)])

In [ ]:
strategy = "cei"  # "cei" or "ei"
strategy = strategy.upper()

# -2 is the default value when no feasible has been found
default_value = -2
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(1, 3)
    train_utility = objective(train_x)
    train_cost = cost(train_x)

    for i in tqdm(range(num_queries)):
        print("query", i)

        feasible_flag = (train_cost <= 0).any()

        if feasible_flag:
            feasible_incumbents[trial, i] = train_utility[train_cost <= 0].max()

        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        cost_model, cost_likelihood = fit_gp_model(train_x, train_cost.squeeze(-1))

        if feasible_flag:
            best_f = train_utility[train_cost <= 0].max()
        else:
            best_f = torch.tensor(default_value)

        if strategy == "EI":
            policy = botorch.acquisition.analytic.ExpectedImprovement(
                model=utility_model,
                best_f=train_utility.max(),
            )

        if strategy == "CEI":
            policy = botorch.acquisition.analytic.ConstrainedExpectedImprovement(
                model=botorch.models.model_list_gp_regression.ModelListGP(
                    utility_model, cost_model
                ),
                best_f=best_f,
                objective_index=0,
                constraints={1: [None, 0]}
            )
        
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

        next_utility = objective(next_x)
        next_cost = cost(next_x)

        print("Test")
        print(train_x)
        print(train_utility)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])
        train_cost = torch.cat([train_cost, next_cost])

In [ ]:
torch.save(feasible_incumbents, f"./{strategy}.pth")
fout = TFile("out.root", "RECREATE")
fout.cd()
for hist in allHistsMC:
    print ("Writing hist: ", hist.GetName())
    hist.Write()
fout.Close()